In [ ]:
import requests
from urllib.parse import unquote
import re

# wiki_link = 'https://ja.wikipedia.org/wiki/%E3%82%AF%E3%83%83%E3%83%91_(%E3%82%B2%E3%83%BC%E3%83%A0%E3%82%AD%E3%83%A3%E3%83%A9%E3%82%AF%E3%82%BF%E3%83%BC)'

def remove_sections(text):
    # 正規表現パターンを作成
    pattern = r"(==\s*脚注\s*==.*?)(?=^==|\Z)|" \
              r"(===\s*注釈\s*===.*?)(?=^==|^===|\Z)|" \
              r"(===\s*出典\s*===.*?)(?=^==|^===|\Z)"

    # セクションを削除
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL | re.MULTILINE)

    return cleaned_text

def get_wikipedia_content(wiki_link):
    title = wiki_link.split('/')[-1]  # URLの最後の部分をタイトルとして使用
    title = unquote(title)  # URLエンコードをデコード

    # MediaWiki APIのエンドポイント
    api_url = 'https://ja.wikipedia.org/w/api.php'

    # APIパラメータの設定
    params = {
        'action': 'query',
        'prop': 'extracts',
        'explaintext': True,   # プレーンテキストで取得
        'format': 'json',
        'titles': title
    }

    # リクエストを送信
    response = requests.get(api_url, params=params)
    if response.status_code != 200:
        print("APIリクエストに失敗しました。")
        return

    data = response.json()
    pages = data['query']['pages']
    page = next(iter(pages.values()))
    if 'extract' in page:
        content = page['extract']
    else:
        print("ページの内容を取得できませんでした。")
        return

    # テキストをファイルに保存
    #不要なセクションを削除
    content = remove_sections(content)
    with open('data/wikipage_' + title+'.txt', 'w', encoding='utf-8') as f:
        f.write(content)

    print(f"コンテンツがwikipage_{title}.txtに保存されました。")

# 使用例
# URLからタイトルを抽出

if __name__ == '__main__':  
    wiki_link = input("WikipediaのページのURLを入力してください：")
    print(f"URL: {wiki_link}")
    get_wikipedia_content(wiki_link)


URL: https://ja.wikipedia.org/wiki/%E8%8B%B1%E9%9B%84%E4%BC%9D%E8%AA%AC_%E8%BB%8C%E8%B7%A1%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA%E3%81%AE%E7%99%BB%E5%A0%B4%E4%BA%BA%E7%89%A9
コンテンツがoutput_英雄伝説_軌跡シリーズの登場人物.txtに保存されました。
